**Nearest Neighbour Distance to the closest Synapse analysis**

**About this script**
> This script takes as an input a compressed zipped file called 'Analysis.zip' containing two directories: Neurites and Synapses. Each directory contains .csv files with different features of neurites or synapses respectively. This script takes the centroids of the synapses and calculates their NND to the closest and to n closest synapses within the associated neurite. It also calculates the mean NNDs and adds them as new columns to the neurite csv information. The output is in a new 'Results' directory and it contains the pooled results per synapses and per neurites in two separate .csv files.

> MIT License

  Copyright (c) Nicolas Peredo

  This script was generated by Nicolas Peredo for Jacqueline Van Vierbergen on the 11/03/2024.

> Permission is hereby granted, free of charge, to any person obtaining a copy
 of this software and associated documentation files (the "Software"), to deal
  in the Software without restriction, including without limitation the rights
  to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
  copies of the Software, and to permit persons to whom the Software is
  furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.









In [1]:
import pandas as pd
from scipy.spatial import distance_matrix
import numpy as np
import os
import zipfile

#Parameters
#Pixel size
pix_size = 0.1992000

#Custumized NND
NNDcustom_number = 3

# Create directories for unzipped files and results
folder_path = '/content/Unzipped'  # Directory for unzipped content
results_path = '/content/Results'  # Directory for results

# Ensuring the directories exist
os.makedirs(folder_path, exist_ok=True)
os.makedirs(results_path, exist_ok=True)

# Path to the zip file
zip_file_path = '/content/Analysis.zip'

# Unzipping the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(folder_path)

# Path to the 'Synapses' subdirectory
centroids_dir = os.path.join(folder_path, 'Analysis', 'Synapses')

# Verify the 'Centroids' directory exists
if not os.path.exists(centroids_dir):
    raise Exception("Centroids directory not found. Please check the zip file structure.")

centroid_files = os.listdir(centroids_dir)

#Empty dataframes to stock the full amount of data
synapses_final_df = pd.DataFrame()
neurites_final_df = pd.DataFrame()

# Process each centroid file
for file in centroid_files:
    file_path = os.path.join(centroids_dir, file)
    synapses_data = pd.read_csv(file_path)

    # Filter data and standardize centroid pixel size
    cleaned_synapses_data = synapses_data[(synapses_data['Axon ID'] != 0) & (synapses_data['Synapse ID'].notna())]
    cleaned_synapses_data['Centroid.X'] = cleaned_synapses_data['Centroid.X'] * pix_size
    cleaned_synapses_data['Centroid.Y'] = cleaned_synapses_data['Centroid.Y'] * pix_size

    #Create empty lists for NND columns
    NND1_synID_column = []
    NNDcustom_synID_column = []

    #Create unique axon value list
    axon_id_list = cleaned_synapses_data['Axon ID'].unique()

    for axon_id in axon_id_list:
      axon_df = cleaned_synapses_data.groupby('Axon ID').get_group(axon_id)
      axon_coordinates = axon_df[['Centroid.X', 'Centroid.Y']].values

      dist_matrix = distance_matrix(axon_coordinates, axon_coordinates)
      np.fill_diagonal(dist_matrix, np.inf)

      syn_id_list = axon_df["Synapse ID"]

      NND1_synID = list(range(0, len(axon_df)))
      NNDcustom_synID = list(range(0, len(axon_df)))

      for syn_id in range(0, len(axon_df)):
        if len(axon_df) < 2:
          NND1_synID[syn_id] = -1
          NNDcustom_synID[syn_id] = -1
          continue
        NND1_synID[syn_id] = sorted(dist_matrix[syn_id,:])[0]
        if (len(axon_df) < NNDcustom_number + 1):
          NNDcustom_synID[syn_id] = -1
          continue
        NNDcustom_synID[syn_id] = sorted(dist_matrix[syn_id,:])[NNDcustom_number-1]

      NND1_synID_column = NND1_synID_column + NND1_synID
      NNDcustom_synID_column = NNDcustom_synID_column + NNDcustom_synID

    cleaned_synapses_data['NND to closest synapse [um]'] = NND1_synID_column
    cleaned_synapses_data['NND to closest ' + str(NNDcustom_number) + ' synapse [um]'] = NNDcustom_synID_column

    # Correct filename manipulation for neurites data
    basename = file.replace('_Synapse_measurements.csv', '')

    # Add a filename column to the dataframe
    cleaned_synapses_data.insert(0, 'Filename', basename)

    # Open neurite data csv
    neurites_file_name = basename + '_Neurites_measurements.csv'
    neurites_path = os.path.join(folder_path, 'Analysis', 'Neurites', neurites_file_name)

    neurites_data = pd.read_csv(neurites_path, encoding='ISO-8859-1')

    # Add the column with the synapse nb/µm as the third column
    neurites_data.insert(3, 'Synapse Nb/um', neurites_data['Synapse_Nb'] / neurites_data['Neurite_length[um]'])

    # Calculate mean NND per Axon
    NND1_synID_median_column = cleaned_synapses_data.groupby('Axon ID').agg({
            'NND to closest synapse [um]': 'mean'
        }).rename(columns={
            'NND to closest synapse [um]': 'Mean NND to closest synapse [um]'
        }).reset_index()

    NNDcustom_synID_median_column = cleaned_synapses_data[(cleaned_synapses_data['NND to closest ' + str(NNDcustom_number) + ' synapse [um]'] != 'NaN')]
    NNDcustom_synID_median_column = NNDcustom_synID_median_column.groupby('Axon ID').agg({
            ('NND to closest ' + str(NNDcustom_number) + ' synapse [um]'): 'mean'
        }).rename(columns={
            ('NND to closest ' + str(NNDcustom_number) + ' synapse [um]'): ('Mean NND to closest ' + str(NNDcustom_number) + ' synapse [um]')
        }).reset_index()

    neurites_merged_df = pd.merge(neurites_data, NND1_synID_median_column, on='Axon ID', how='left')
    neurites_merged_df = pd.merge(neurites_merged_df, NNDcustom_synID_median_column, on='Axon ID', how='outer')

    # Add a filename column to the dataframe
    neurites_merged_df.insert(0, 'Filename', basename)

    # Use concat to add the dataframes together
    synapses_final_df = pd.concat([synapses_final_df, cleaned_synapses_data], ignore_index=True)
    neurites_final_df = pd.concat([neurites_final_df, neurites_merged_df], ignore_index=True)

    # Save the results
    savingpath_synapses = os.path.join(results_path, 'Pooled_results_synapses.csv')
    savingpath_neurites = os.path.join(results_path, 'Pooled_results_neurites.csv')

    synapses_final_df.to_csv(savingpath_synapses, index=False)
    neurites_final_df.to_csv(savingpath_neurites, index=False)
